Install Required Libraries 


In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
!pip install spectralcluster
from spectralcluster import SpectralClusterer

In [3]:
import numpy as np

In [ ]:
Mounting Drive(Kindly change locations according to file locations)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


Making test data batch size equal to that required(Batchsize = 8 has been used)


In [65]:
X_test = np.load('/content/gdrive/MyDrive/ML Datasets/TS3010c.npy')
test_len = len(X_test)

In [66]:
X_test.shape

(1136, 256)

In [67]:
val = int(np.ceil((len(X_test)/8))*8 - len(X_test))
X_test = np.append(X_test,X_test[len(X_test)-val:len(X_test)],axis = 0)

In [75]:
np.save('/content/gdrive/MyDrive/ML Datasets/test_1.npy',X_test)

In [8]:
%matplotlib inline

Running GANMM to get the 1.Sampled data 2. Clusters

In [ ]:
#Please change the file address in tflib/sae_minist.py before executing

In [ ]:
%run "/content/gdrive/MyDrive/ML Datasets/GANMM-master/main.py" sae_mnist

Augmenting fake data to balance out clusters

In [46]:
fake1 = np.load('/content/gdrive/MyDrive/ML Datasets/fakeData.npy')

In [47]:
fake1.shape

(4, 248, 256)

In [48]:
Y_pred = np.load('/content/gdrive/MyDrive/ML Datasets/GANMM-master/GanmmPred.npy')

In [49]:
Y_pred.shape

(1136,)

Deciding How much to add data

In [68]:
(unique, counts) = np.unique(Y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[  0, 214],
       [  1, 353],
       [  2, 261],
       [  3, 308]])

In [69]:
n_cluster = 4

In [70]:
add_num = []
for i in range(0,n_cluster):
  add_num.append(0)
avg = int(len(X_test)/n_cluster )
for i in range(0,n_cluster):
  if(avg-frequencies[i][1] > 0):
    add_num[i] = min(max(avg-frequencies[i][1],0),150)
  

In [71]:
len(X_test)

1136

Finally adding this to X_test and saving

In [72]:
add_num #this shows the ammount of data augmeneted to each cluster

[70, 0, 23, 0]

In [73]:
for i in range(0,n_cluster):
  if(add_num[i]>0):
    X_test = np.append(X_test,fake1[i][:add_num[i]],axis = 0)

Using Spectral Clustering On this data(X_test)

In [63]:
X_test.shape

(1368, 256)

In [ ]:
#After this step we run spectral clustering in balanced dataset

In [64]:
clusterer = SpectralClusterer(
                  min_clusters=4,
                  max_clusters=4,
                  p_percentile=0.90,
                  gaussian_blur_sigma=0)
curr_labels = clusterer.predict(X_test)
np.save('/content/gdrive/MyDrive/Advanced ML project 2/GANMM preds/TS3010c.npy',curr_labels[:test_len])